# LibTNN 工具包的使用

LibTNN 是一个张量工具包，提供了以下几种功能：

- `CP_TRL(CP Tensor Regression Layer)`           CP分解张量回归层

- `Tucker_TRL(Tucker Tensor Regression Layer)`   Tucker分解张量回归层

- `TCL(Tensor Contraction Layer)`                张量收缩层

- `CP_CNN(CP SPEED-UP CNN)`                      CP分解加速的CNN层（函数）

- `Tucker_CNN(Tucker SPEED-UP CNN)`              Tucker分解加速的CNN层（函数）

- `cp_decomposition_conv_layer`                  使用CP分解加速CNN层

- `tucker_decomposition_conv_layer`              使用Tucker分解加速的CNN层

目前本工具包提供的层能够与 PyTorch 兼容，并实现训练

使用其他包与工具：

- `PyTorch`

- `torchvision`

- `Tensorly`

- `NumPy`

Author: YB.Li

首先载入相关包

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

import tensorly as tl
from tensorly.tenalg import inner

import numpy as np

导入 LibTNN 的相关部件

In [2]:
from LibTNN import CP_TRL, CP_CNN, TCL, Tucker_CNN, Tucker_TRL

设定相关参数与运算后端

In [3]:
batch_size = 16

if torch.cuda.is_available:
    device = 'cuda:0'
else:
    device = 'cpu'
    
tl.set_backend('pytorch')

加载 MINIST 数据集

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data/', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

定义一个函数，用来统计网络的参数数目

In [5]:
def count_params(network):
    return np.sum(np.prod(p.size()) for p in network.parameters())

# 原始的 CNN MNIST 手写体网络

传统的 CNN 网络包括卷积层，全连接层，最后通过 Softmax 来实现概率回归

下面呢，我们定义一个 Classic_CNN ：

In [6]:
class Classic_CNN(nn.Module):
    
    def __init__(self):
        
        super(Classic_CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 50, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(800, 50)
        self.fc2 = nn.Linear(800, 10)

    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 800)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x)

实例化一个 Classic_CNN，并统计一下 Classic_CNN 的参数数目(61130)

In [7]:
My_Classic_CNN = Classic_CNN()
parameter_num_Classic_CNN = count_params(My_Classic_CNN)
print(parameter_num_Classic_CNN)

61130


C:\Users\beran\AppData\Local\Temp/ipykernel_18240/3053006230.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.prod(p.size()) for p in network.parameters())


# TCL_Tucker_TRL 手写体识别网络

本网络将演示 TCL 与 Tucker_TRL 的使用

该网络将在两层卷积网络后，用 TCL 代替 Classic_CNN 中的 fc1(全连接层)，Tucker_TRL 代替 Classic_CNN 中的 fc2(回归层)

In [8]:
class TCL_Tucker_TRL_Net(nn.Module):
    
    def __init__(self):
        
        super(TCL_Tucker_TRL_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.norm = nn.BatchNorm2d(50)
        self.tcl = TCL(input_size=(batch_size, 50, 4, 4), output_size=(batch_size, 5, 5, 2))
        self.trl = Tucker_TRL(ranks=(5, 3, 2, 2), input_size=(batch_size, 5, 2, 2), output_size=(1, 10))
        
    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.norm(x)
        x = self.tcl(x)
        x = self.trl(x)
        
        return F.log_softmax(x)

实例化一个 TCL_Tucker_TRL_CNN，并统计一下网络的参数数目(26064)

In [9]:
My_TCL_Tucker_TRL_Net= TCL_Tucker_TRL_Net()
parameter_num_TCL_Tucker_TRL_Net = count_params(My_TCL_Tucker_TRL_Net)
print(parameter_num_TCL_Tucker_TRL_Net)

26064


C:\Users\beran\AppData\Local\Temp/ipykernel_18240/3053006230.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.prod(p.size()) for p in network.parameters())


TCL(Tensor Contraction Layer)层的实现

实现思想：

一般的深度卷积神经网络在使用卷积层进行特征提取后，得到的激活张量会通过 Flatten 层进行展平，然后使用全连接来实现输出，但是 Flatten 层会破坏张量的空间结构，全连接层的参数也过多，因此引入 TCL 层

实现思想`（单样本情况）`：

对于一个大小为 $ (1, m_1, m_2, m_3) $ 大小的激活张量，希望得到一个大小为 $ (1, n_1, n_2, n_3) $ 的输出作为下一层的输入
可以使用模态积 (model-dot) 的方法，使用三个大小为 $ (n_1, m_1) (n_2, m_2) (n_3, m_3) $ 的矩阵，分别进行 `mode-1` `mode-2` `mode-3` 的模态积

参数比较：

对于一个大小为 $ (1, m_1, m_2, m_3) $ 大小的激活张量，得到一个大小为 $ (1, n_1, n_2, n_3) $ 的输出作为下一层的输入，若使用全连接层，那么参数量将为

$$
\overset3{\underset{k=0}{\prod m_{(k)}}}\overset3{\underset{k=0}{\prod n_{(k)}}}
$$

使用 TCL 层后，需要的参数仅为三个矩阵，大小分别为 $ (n_1, m_1) (n_2, m_2) (n_3, m_3) $，那么参数量将为：

$$
\overset3{\underset{k=0}{\sum m_{(k)}n_{(k)}}}
$$

看见没，由连乘变为连加咯！！！

对于一个大小为 (batch_size, m1, m2, m3) 大小的激活张量，希望得到一个大小为 (batch_size, n1, n2, n3) 的输出作为下一层的输入
，可以使用模态积的方法，使用三个大小为 (n1, m1) (n2, m2) (n3, m3) 的矩阵，分别进行 mode-1 mode-2 mode-3 的模态积，可以用来代替展平层和全连接层

TCL.__init__(self, input_size: tuple, output_size: tuple, verbose=1, **kwargs):

- input_size: tuple   为输入激活张量的大小 (batch_size, m1, m2, m3)，其第一个参数 batch_size 无效

- output_size: tuple  为输出矩阵的大小 (batch_size, class_num)，其第一个参数 batch_size 无效


self.tcl = TCL(input_size=(batch_size, 50, 4, 4), output_size=(batch_size, 5, 5, 2))

TCL 层的第一个参数为输入矩阵大小将，接收上一层的输入，故 `input_size=(batch_size, 50, 4, 4)`， TCL 层的第二个参数为输出矩阵大小，为了与 Clasic_CNN 中的全连接层参数相同，故 `output_size=(batch_size, 5, 2, 2)`，相当于实现了 `50 * 4 * 4 (800)` 到 `5 * 5 * 2 (50)` 的全连接

Tucker_TRL(Tucker Tensor Regression Layer)层的实现

实现思想

对于一个大小为 $ (batch_size, m_1, m_2, m_3)  $ 大小的激活张量，网络回归希望得到一个大小为 $ (batch_size, class_num) $ 的输出

为了得到和预期输出，可以使用一个大小为 $ (m_1, m_2, m_3, class_num) $ 的回归权重张量，采用广义内积 (Generalized inner-product) 的形式，得到输出

回归权重张量的大小过大，不便于计算，可以使用 Tucker 分解的思想，使用秩 $ (a, b, c, d) $ 的 Tucker 分解

核心张量大小为 $ (a, b, c) $，分解矩阵大小分别为 $ (m_1, a) $ $ (m_2, b) $ $ (m_3, c) $ $ (class_num, d) $，简化计算

Tucker_TRL.__init__(self, input_size: tuple, output_size: tuple, ranks: tuple, verbose=1, **kwargs)

- input_size: tuple   为输入激活张量的大小 (batch_size, m1, m2, m3)，其第一个参数 batch_size 无效

- output_size: tuple  为输出矩阵的大小 (batch_size, class_num)，其第一个参数 batch_size 无效

- ranks: tuple        为回归张量 Tucker 分解核心张量大小

# TCL_CP_TRL 手写体识别网络

本网络将演示 TCL 与 CP_TRL 的使用

该网络将在两层卷积网络后，用 TCL 代替 Classic_CNN 中的 fc1(全连接层)，CP_TRL 代替 Classic_CNN 中的 fc2(回归层)

In [10]:
class TCL_CP_TRL_Net(nn.Module):
    
    def __init__(self):
        
        super(TCL_CP_TRL_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.norm = nn.BatchNorm2d(50)
        self.tcl = TCL(input_size=(batch_size, 50, 4, 4), output_size=(batch_size, 5, 5, 2))
        self.trl = CP_TRL(input_size=(batch_size, 5, 2, 2), output_size=(1, 10), rank=3)
        
    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.norm(x)
        x = self.tcl(x)
        x = self.trl(x)
        
        return F.log_softmax(x)

实例化一个 TCL_CP_TRL_Net，并统计一下网络的参数数目(26009)

In [12]:
My_TCL_CP_TRL_Net = TCL_CP_TRL_Net()
parameter_num_TCL_CP_TRL_Net = count_params(My_TCL_CP_TRL_Net)
print(parameter_num_TCL_CP_TRL_Net)

26009


C:\Users\beran\AppData\Local\Temp/ipykernel_18240/3053006230.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.prod(p.size()) for p in network.parameters())


CP_TRL(Tensor Regression Layer) 层的实现

实现思想:

对于一个大小为 $ (batch_size, m_1, m_2, m_3) $ 大小的激活张量，网络回归希望得到一个大小为 $ (1, class_num) $ 的输出

为了得到和预期输出，可以使用一个大小为 $ (m1, m2, m3, class_num) $ 的回归权重张量，采用广义内积 (Generalized inner-product) 的形式，得到输出

回归权重张量的大小过大，不便于计算，可以使用 CP 分解的思想，使用秩 R 的 CP 分解

权重系数矩阵大小为 $ (R, 1) $，分解矩阵大小分别为 $ (m1, R) $ $ (m2, R) $ $ (m3, R) $ $ (class_num, R) $，简化计算

CP_TRL.__init__(self, input_size: tuple, output_size: tuple, rank: int, verbose=1, **kwargs)

- input_size: tuple   为输入激活张量的大小 (batch_size, m1, m2, m3)，其第一个参数 batch_size 无效

- output_size: tuple  为输出矩阵的大小 (batch_size, class_num)，其第一个参数 batch_size 无效

- rank: int           为回归张量 CP 分解的秩

# Tucker_CNN_TCL_Tucker_TRL 手写体识别网络

本网络将演示 Tucker_CNN 的使用

该网络将 使用Tucker_CNN 代替普通的卷积，用 TCL 代替 Classic_CNN 中的 fc1(全连接层)，Tucker_TRL 代替 Classic_CNN 中的 fc2(回归层)

In [16]:
class Tucker_CNN_TCL_Tucker_TRL_Net(nn.Module):
    
    def __init__(self):
        
        super(Tucker_CNN_TCL_Tucker_TRL_Net, self).__init__()
        self.conv1 = Tucker_CNN(1, 20, 5, (1, 10))
        self.conv2 = Tucker_CNN(20, 50, 5, (10, 25))
        self.norm = nn.BatchNorm2d(50)
        self.tcl = TCL(input_size=(batch_size, 50, 4, 4), output_size=(batch_size, 5, 5, 2))
        self.trl = Tucker_TRL(ranks=(5, 3, 2, 2), input_size=(batch_size, 5, 2, 2), output_size=(1, 10))
        
    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.norm(x)
        x = self.tcl(x)
        x = self.trl(x)
        
        return F.log_softmax(x)

实例化一个 Tucker_CNN_TCL_Tucker_TRL_Net，并统计一下网络的参数数目(8715)

In [17]:
My_Tucker_CNN_TCL_Tucker_TRL_Net = Tucker_CNN_TCL_Tucker_TRL_Net()
parameter_num_Tucker_CNN_TCL_Tucker_TRL_Net = count_params(My_Tucker_CNN_TCL_Tucker_TRL_Net)
print(parameter_num_Tucker_CNN_TCL_Tucker_TRL_Net)

8715


C:\Users\beran\AppData\Local\Temp/ipykernel_18240/3053006230.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.prod(p.size()) for p in network.parameters())


Tucker_CNN 层的实现

实现思想:

对于一个卷积核，可以在输入通道和输出通道方向，进行部分 Tucker 分解，将卷积运算简化为三部，可以概括为

- 输入通道压缩

- 小规模常规卷积

- 输出通道恢复

Tucker_CNN.__init__(self, in_channels: int, out_channels: int, kernel_size: int, ranks: tuple, device='cpu', **kwargs)

- in_channels:  int     输入通道数

- out_channels: int     输出通道数

- kernel_size:  int     模拟卷积核大小

- ranks:        tuple   降维参数

- ranks[0]为降维输入通道数

- ranks[1]为降维输出通道数

注意，使用该层后，就可以不采用 BN 层了

# CP_CNN_TCL_CP_TRL 手写体识别网络

本网络将演示 CP_CNN 的使用

该网络将 使用 CP_CNN 代替普通的卷积，用 TCL 代替 Classic_CNN 中的 fc1(全连接层)，CP_TRL 代替 Classic_CNN 中的 fc2(回归层)

In [18]:
class CP_CNN_TCL_CP_TRL_Net(nn.Module):
    
    def __init__(self):
        
        super(CP_CNN_TCL_CP_TRL_Net, self).__init__()
        self.conv1 = CP_CNN(1, 20, 5, 10)
        self.conv2 = CP_CNN(20, 50, 5, 16)
        self.norm = nn.BatchNorm2d(50)
        self.tcl = TCL(input_size=(batch_size, 50, 4, 4), output_size=(batch_size, 5, 5, 2))
        self.trl = CP_TRL(rank=4, input_size=(batch_size, 5, 2, 2), output_size=(1, 10))
        
    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.norm(x)
        x = self.tcl(x)
        x = self.trl(x)
        
        return F.log_softmax(x)

实例化一个 CP_CNN_TCL_CP_TRL_Net，并统计一下网络的参数数目(2119)

In [19]:
My_CP_CNN_TCL_CP_TRL_Net = CP_CNN_TCL_CP_TRL_Net()
parameter_num_CP_CNN_TCL_CP_TRL_Net = count_params(My_CP_CNN_TCL_CP_TRL_Net)
print(parameter_num_CP_CNN_TCL_CP_TRL_Net)

2119


C:\Users\beran\AppData\Local\Temp/ipykernel_18240/3053006230.py:2: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  return np.sum(np.prod(p.size()) for p in network.parameters())


CP_CNN 层的实现

实现思想:

对于一个卷积核，可以进行 CP 分解，将卷积运算简化为四步部，可以概括为

- 输入通道压缩

- 水平方向逐通道卷积

- 垂直方向逐通道卷积

- 输出通道恢复

CP_CNN.__init__(self, in_channels: int, out_channels: int, kernel_size: int, rank: int, device='cpu', **kwargs)

- n_channels:  int     输入通道数

- out_channels: int     输出通道数

- kernel_size:  int     模拟卷积核大小

- rank:         int     降维参数
